In [ ]:
import matplotlib.pyplot as plt
from razredi import Tocka
from jarvis_march import grid_peel_jarvis_enakomerna
from graham_scan import grid_peel_graham_enakomerna

In [ ]:
graham = grid_peel_graham_enakomerna(3,3)
jarvis = grid_peel_jarvis_enakomerna(5,5)
print(jarvis)
#print(graham)

In [ ]:
graham_1 = [(i.x, i.y) for i in graham[1][0]]
graham_2 = [(i.x, i.y) for i in graham[1][1]]
graham_3 = [(i.x, i.y) for i in graham[1][2]]





In [ ]:
plt.plot(*zip(*graham_1), ".r-")
plt.plot(*zip(*graham_2), ".b-")
plt.plot(*zip(*graham_3), "-o")
